In [ ]:
import json
from typing import List, Dict
from pydantic import BaseModel, Field

# --- Define a simple protein selection logic ---
class ProteinSelectionAgent:
    def select_best_protein(self, proteins: List[Dict[str, str]], target_function: str) -> Dict[str, str]:
        # For demonstration, we perform a simple case-insensitive match.
        matching = [p for p in proteins if p["function"].lower() == target_function.lower()]
        if matching:
            return matching[0]
        # Fallback: choose the first protein if no exact match is found.
        return {"protein": proteins[0]["protein"], "function": proteins[0]["function"]}

# --- Define the input schema for the tool ---
class ProteinSelectionInput(BaseModel):
    proteins: List[Dict[str, str]] = Field(..., description="List of proteins with their functions")
    target_function: str = Field(..., description="The desired protein function")

# --- Create a custom CrewAI tool ---
from crewai_tools import BaseTool

class ProteinSelectionTool(BaseTool):
    name: str = "ProteinSelectionTool"
    description: str = "Selects the best protein from a list based on the target function."
    args_schema: type = ProteinSelectionInput

    def _run(self, proteins: List[Dict[str, str]], target_function: str) -> str:
        selector = ProteinSelectionAgent()
        best = selector.select_best_protein(proteins, target_function)
        return json.dumps(best, indent=2)

    async def _arun(self, proteins: List[Dict[str, str]], target_function: str) -> str:
        raise NotImplementedError("Asynchronous execution is not implemented.")

# --- Define the CrewAI agent using our custom tool ---
from crewai import Agent, Task, Crew, Process

protein_selector_agent = Agent(
    role="Protein Selection Expert",
    goal="Select the best protein that matches the given function requirement from a provided list.",
    backstory="You are an expert in protein biochemistry and bioinformatics, with a keen ability to match protein functions to desired roles.",
    verbose=True,
    tools=[ProteinSelectionTool()]
)

# --- Create a task that leverages this agent ---
selection_task = Task(
    description="Choose the best protein from the provided list that performs the target function.",
    expected_output="A JSON object detailing the selected protein and its function.",
    agent=protein_selector_agent,
)

# --- Assemble the crew and run the task ---
crew = Crew(
    agents=[protein_selector_agent],
    tasks=[selection_task],
    process=Process.sequential,
    verbose=True,
)

# --- Define input: a list of protein-function pairs and a target function ---
proteins = [
    {"protein": "ProteinA", "function": "Enzymatic catalysis"},
    {"protein": "ProteinB", "function": "Structural support"},
    {"protein": "ProteinC", "function": "Signal transduction"}
]
target_function = "Signal transduction"

inputs = {
    "proteins": proteins,
    "target_function": target_function
}

result = crew.kickoff(inputs=inputs)
print("Best matching protein:", result.raw)